In [1]:
import re
import json
import random
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import numpy as np
import pandas as pd
import IPython.display as ipd
import soundfile as sf

import torch
from datasets import load_dataset, load_metric
from transformers import (Wav2Vec2CTCTokenizer, 
                          Wav2Vec2FeatureExtractor, 
                          Wav2Vec2Processor, 
                          Wav2Vec2ForCTC,
                          TrainingArguments,
                          Trainer)

In [2]:
timit = load_dataset("timit_asr")
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

print(timit)

Reusing dataset timit_asr (/home/rocabrera/.cache/huggingface/datasets/timit_asr/clean/2.0.1/66d672b7070257726bc9e76100d06d3e342aa6c37aed742803302c293540e96d)


DatasetDict({
    train: Dataset({
        features: ['file', 'text'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'text'],
        num_rows: 1680
    })
})


In [5]:
isinstance(timit, torch.utils.data.Dataset)

False

In [3]:
# n random samples
def show_samples(dataset, qtd_random = 10):
    random_instances = [random.randint(0, dataset.num_rows) for _ in range(qtd_random)]
    ipd.display(pd.DataFrame(dataset[random_instances]))
    
show_samples(timit["train"].remove_columns(["file"]))

,text
0,Most young rise early every morning.
1,The full moon shone brightly that night.
2,Westchester is a county in New York.
3,Her auburn hair reminded him of autumn leaves.
4,Orange juice tastes funny after toothpaste.
5,Nonprofit organizations have frequent fund rai...
6,But now she sank lower to the rock.
7,Remember to allow identical twins to enter fre...
8,The most recent geological survey found seismi...
9,The news agency hired a great journalist.


In [4]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

timit = timit.map(remove_special_characters)

In [5]:
show_samples(timit["train"].remove_columns(["file"]))

,text
0,alfalfa is healthy for you
1,allow each child to have an ice pop
2,the guerrilla bivouac remained silent
3,do you have the yellow ointment ready
4,junior what on earth's the matter with you
5,employee layoffs coincided with the company's ...
6,count the number of teaspoons of soysauce that...
7,butterscotch fudge goes well with vanilla ice ...
8,they are not true because scientists or prophe...
9,coconut cream pie makes a nice dessert


In [6]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = timit.map(extract_all_chars,
                   batched=True,
                   batch_size=-1, 
                   keep_in_memory=True, 
                   remove_columns=timit.column_names["train"])

In [7]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'i': 0,
 "'": 1,
 'h': 2,
 'g': 3,
 'm': 4,
 'f': 5,
 'u': 6,
 'c': 7,
 'w': 8,
 'e': 9,
 'l': 10,
 'o': 11,
 'n': 12,
 't': 13,
 'z': 14,
 'b': 15,
 'x': 16,
 'd': 17,
 'p': 18,
 'k': 19,
 ' ': 20,
 'r': 21,
 's': 22,
 'y': 23,
 'q': 24,
 'a': 25,
 'v': 26,
 'j': 27}

In [8]:
"""
Trocando ' ' por '|'
"""

vocab_dict["|"] = vocab_dict[" "]
vocab_dict.pop(" ")

20

In [9]:
"""
Adiciona o blank token (componente core no algoritmo do CTC). 
"""
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

In [10]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [11]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", 
                                 unk_token="[UNK]", 
                                 pad_token="[PAD]", 
                                 word_delimiter_token="|")

In [12]:
"""
É importante saber o sampling_rate do embedding onde os embeddings foram pré treinados.
"""

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, 
                                             sampling_rate=16000, 
                                             padding_value=0.0, 
                                             do_normalize=True, 
                                             return_attention_mask=False)

In [15]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [16]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["file"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

timit = timit.map(speech_file_to_array_fn, remove_columns=timit.column_names["train"], num_proc=4)

In [17]:
rand_int = random.randint(0, len(timit["train"]))

ipd.Audio(data=np.asarray(timit["train"][rand_int]["speech"]), autoplay=True, rate=16000)

In [18]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["speech"]).shape)
print("Sampling rate:", timit["train"][rand_int]["sampling_rate"])

Target text: most young rise early every morning
Input array shape: (34714,)
Sampling rate: 16000


In [19]:
"""
Vai precisar entender e adaptar:

https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__
"""

def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

timit_prepared = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], batch_size=8, num_proc=4, batched=True)

/home/rocabrera/.pyenv/versions/venv_ped/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/home/rocabrera/.pyenv/versions/venv_ped/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/home/rocabrera/.pyenv/versions/venv_ped/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested s

/home/rocabrera/.pyenv/versions/venv_ped/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/home/rocabrera/.pyenv/versions/venv_ped/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/home/rocabrera/.pyenv/versions/venv_ped/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested s

In [20]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch
    
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [21]:
wer_metric = load_metric("wer")

In [23]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [24]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base", 
                                       gradient_checkpointing=True, 
                                       ctc_loss_reduction="mean", 
                                       pad_token_id=processor.tokenizer.pad_token_id,)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [25]:
training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-base-timit-demo",
  output_dir="./wav2vec2-base-timit-demo",
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,)

In [26]:
trainer = Trainer(model=model,
                  data_collator=data_collator,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=timit_prepared["train"],
                  eval_dataset=timit_prepared["test"],
                  tokenizer=processor.feature_extractor,)

In [27]:
torch.cuda.is_available()

True

In [28]:
trainer.train()

/home/rocabrera/.pyenv/versions/venv_ped/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
500,3.803900,3.046154,0.998070,88.514400,18.980000
1000,1.453400,0.665251,0.539315,89.264500,18.820000
1500,0.597800,0.497777,0.433740,90.033500,18.660000
2000,0.405900,0.468680,0.400662,89.914400,18.684000
2500,0.290700,0.454023,0.376266,89.102400,18.855000
3000,0.264900,0.480012,0.363586,89.209300,18.832000
3500,0.209500,0.465324,0.350286,88.531500,18.976000
4000,0.187300,0.418406,0.351389,88.477800,18.988000
4500,0.184600,0.444803,0.336090,88.914300,18.895000


KeyboardInterrupt: 

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))